In [1]:
import torch
import torch.nn as nn
import torch.nn.function as F
import sys
import os
import torch.utils.model_zoo as model_zoo

class basicConv2d(nn.module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding=0, bias):
        super(BasicConv2d, self).__init__()
        self.conv=nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride, padding=0, bias=False)
        self.bnorm=nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0, affline=True)
        self.relu=nn.ReLU(inplace=False)
    
    def forward(self, x):
        x=self.conv(x)
        x=self.bnorm(x)
        x=self.relu(x)
        return x

class mixed1a(nn.module):
    def __init__(self):
        super(mixed1a, self).__init__()
        self.branch1=basicConv2d(192, 96, kernel_size=1, stride=1)
        self.branch2=nn.sequential(
            basicConv2d(192, 64, kernel_size=1, stride=1), #192, 48
            basicConv2d(64, 96, kernel_size=3, stride=1, padding=2) # 48, 64, kernel_size=5, doubt in padding
        )
        self.branch3=nn.sequential(
            basicConv2d(192, 64, kernel_size=1, stride=1),
            basicConv2d(64, 96, kernel_size=3, stride=1, padding=1), #doubt in padding
            basicConv2d(96, 96, kernal_size=3, stride=1, padding=1)
        )
        self.branch4=nn.sequential(
            nn.AvgPool(3, stride=1, padding=1, count_include_pad=False), #doubt
            basicConv2d(192, 96, kernel_size=1, stride=1)
        )
        
    def forward(self, x):
        x1=self.branch1(x)
        x2=self.branch2(x)
        x3=self.branch3(x)
        x4=self.branch4(x)
        output=torch.cat((x1, x2, x3, x4), 1)
        return output

class Block1(nn.Module):
    def __init__(self, scale=1.0):
        super(Block1, self).__init__()
        self.scale=scale
        self.branch1=basicConv2d(384, 32, kernel_size=1, stride=1) #320
        self.branch2=nn.Sequential(
            basicConv2d(384, 32, kernel_size=1, stride=1), #320
            basicConv2d(32, 32, kernel_size=3, stride=1, padding=1) #padding 
        )
        self.branch3=nn.Sequential(
            basicConv2d(384, 32, kernel_size=1, stride=1),
            basicConv2d(32, 48, kernel_size=3, stride=1, padding=1),
            basicConv2d(48, 64, kernel_size=3, stride=1, padding=1)
        )
        self.conv2d=nn.Conv2d(128, 384, kernel_size=1, stride=1) #320
        self.relu=nn.ReLU(inplace=False) #inplace?
    def forward(self, x):
        x1=self.branch1(x)
        x2=self.branch2(x)
        x3=self.branch3(x)
        output=torch.cat((x1, x2, x3), 1)
        output=self.conv2d(output)
        output=output*self.scale+x
        output=self.relu(output)
        return output

class mixed2a(nn.module):
    def __init__(self):
        super(mixed2a, self).__init__()
        self.branch1=basicConv2d(384, 384, kernel_size=3, stride=2) #320
        self.branch2=nn.sequential(
            basicConv2d(384, 256, kernel_size=1, stride=1),
            basicConv2d(256, 256, kernel_size=3, stride=1, padding=1), #??
            basicConv2d(256, 384, kernel_size=3, stride=2)
        )
        self.branch3=nn.MaxPool2d(3, stride=2)
    def forward(self, x):
        x1=self.branch1(x)
        x2=self.branch2(x)
        x2=self.branch3(x)
        output=torch.cat((x1, x2, x3), 1)
        return output
    
class Block2(nn.Module):
    def __init__(self, scale=1.0):
        super(Block2, self).__init__()
        self.scale=scale
        self.branch1=basicConv2d(1088, 192, kernel_size=1, stride=1) #how 1088?
        self.branch2=nn.Sequential(
            basicConv2d(1088, 128, kernel_size=1, stride=1),
            basicConv2d(128, 160, kernel_size=(1, 7), stride=1, padding=(0, 3)), #padding?
            basicConv2d(160, 192, kernel_size=(7, 1), stride=1, padding=(0, 3))
        )
        self.conv2d=nn.Conv2d(384, 1154, kernel_size=1, stride=1) #1088
        self.relu=nn.ReLU(inplace=False)
    def forward(self, x):
        x1=self.branch1(x)
        x2=self.branch2(x)
        output=torch.cat((x1, x2), 1)
        output=self.conv2d(output)
        output=output*self.scale+x
        output=self.relu(output)
        return output
        
class mixed3a(nn.module):
    def __init__(self):
        super(mixed3a, self).__init__()
        self.branch1=nn.Sequential(
            basicConv2d(1154, 256, kernel_size=1, stride=1), #1088
            basicConv2d(256, 384, kernel_size=3, stride=2)
        )
        self.branch2=nn.Sequential(
            basicConv2d(1154, 256, kernel_size=1, stride=1), #1088
            basicConv2d(256, 288, kernel_size=3, stride=2)
        )
        self.branch3=nn.Sequential(
            basicConv2d(1154, 256, kernel_size=1, stride=1), #1088
            basicConv2d(256, 288, kernel_size=3, stride=1, padding=1),
            basicConv2d(288, 320, kernel_size=3, stride=2)
        )
        self.branch4=nn.MaxPool2d(3, stride=2)
    def forward(self, x):
        x1=self.branch1(x)
        x2=self.branch2(x)
        x3=self.branch3(x)
        x4=self.branch4(x)
        output=torch.cat((x1, x2, x3, x4), 1)
        return output

class Block3(nn.Module):
    def __init__(self, scale=1.0, noReLu=False):
        super(Block3, self).__init__()
        self.scale=scale
        self.noReLU=noReLu
        self.branch1=basicConv2d(2146, 192, kernel_size-1, stride=1) #2080 (384+288+320+1154)
        self.branch2=nn.Sequential(
            basicConv2d(2146, 192, kernel_size=1, stride=1),
            basicConv2d(192, 224, kernel_size=(1,3), stride=1, padding=(0, 1)),
            basicConv2d(224, 256, kernel_size=(3,1), stride=1, padding=(1,0))
        )
        self.conv2d=nn.Conv2d(448, 2048, kernel_size=1, stride=1) #2080
        if not self.noReLU:
            self.relu=nn.ReLU(inplace=False)
    def forward(self, x):
        x1=self.branch1(x)
        x2=self.branch2(x)
        output=torch.cat((x1, x2), 1)
        output=self.conv2d(output)
        output=output*self.scale+x
        if not self.noReLU:
            output=self.relu(output)
        return output
            
        
class inceptionResnetV2(nn.module):
    def __init__(self, classes=50):
        super(inceptionResnetV2, self).__init__()
        self.conv1=basicConv2d(3, 32, kernel_size=3, stride=2)
        self.conv2=basicConv2d(32, 32, kernel_size=3, stride=1)
        self.conv3=basicConv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.maxPool1=nn.MaxPool2d(3, stride=2)
        self.conv4=basicConv2d(64, 80, kernel_size=1, stride=1)
        self.conv5=basicConv2d(80, 192, kernal_size=3, stride=1)
        self.maxPool2=nn.MaxPool2d(3, stride=2)
        self.mixed1=mixed1a()
        self.repeat1=nn.Sequential(
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17),
            Block1(scale=0.17)
        )
        self.mixed2=mixed2a()
        self.repeat2=nn.Sequential(
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10),
            Block2(scale=0.10)
        )
        self.mixed3=mixed3a()
        self.repeat3=nn.Sequential(
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20),
            Block3(scale=0.20)
        )
        self.block3=Block3(noReLU=True)
        self.conv6=basicConv2d(2048, 1792, kernel_size=1, stride=1) #2080, 1536
        self.avgpool1=nn.AvgPool2d(8, count_include_pad=False)
        self.classif=nn.Linear(1792, classes) #1536
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.maxPool1(x)
        x=self.conv4(x)
        x=self.conv5(x)  
        x=self.maxPool2(x)
        x=self.mixed1(x)
        x=self.repeat1(x)
        x=self.mixed2(x)
        x=self.repeat2(x)
        x=self.mixed3(x)
        x=self.repeat3(x)
        x=self.block3(x)
        x=self.conv6(x)
        x=self.avgpool1(x)
        x=x.view(x.size(0), -1) #doubt
        x=self.classif(x)
        return x

SyntaxError: non-default argument follows default argument (<ipython-input-1-2ef6a48b26b3>, line 9)